# 1. 導入套件

In [1]:
# 主要套件
import torch

# 資料處理
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 建立機器模型
from torch import nn
from torch import optim

# 2. 修正參數

In [2]:
# 擷取資料檔案
gesture_data_file = "dataset/train_data-x_y.csv"

In [3]:
# 取得 GPU 或是 CPU 的設備進行訓練
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
# 批次
batch_size = 16

In [5]:
# 優化函數
optim_fn = optim.SGD
# 損失函數
loss_fn = nn.CrossEntropyLoss()

In [6]:
# 學習率
learning_rate = 1e-5
# 世代
epochs = 30

# 3. 類別

In [7]:
# 姿態資料集
class GestureDataset(Dataset):
    def __init__(self, df:pd.DataFrame):
        self.dataframe = df
        pass
    
    def __getitem__(self, idx):
        # 取得資料特徵，轉換成 torch 格式，資料為 torch.float32 格式
        feature = torch.tensor(self.dataframe.iloc[idx, :-1]).to(torch.float32)
        # 取得結果
        result = self.dataframe.iloc[idx, -1]
        return feature, result
    
    def __len__(self):
        return len(self.dataframe)

In [8]:
# 姿態模型
class GestureModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.relu_linear_stack = nn.Sequential(
            nn.Linear(34, 50),
            nn.ReLU(),
            nn.Linear(50, 4)
        )
        pass
    
    def forward(self, x):
        logits = self.relu_linear_stack(x)
        return logits

# 4. 函數

In [9]:
# 訓練函數
def train(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module, optimizer:optim.Optimizer):
    size = len(dataloader.dataset)
    
    # 開始訓練
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 轉換成可讓模型訓練的格式
        X, y = X.to(device), y.to(device)
        
        # 計算預測誤差
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 輸出
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [10]:
# 測試函數
def test(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module):
    
    # 計算結果參數
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 測試誤差、正確率
    test_loss, correct = 0, 0
    
    # 開始驗證
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            # 轉換成可讓模型訓練的格式
            X, y = X.to(device), y.to(device)

            # 計算預測誤差
            pred = model(X)
            loss = loss_fn(pred, y).item()
            
            # 計算誤差、正確率
            test_loss += loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
# 輔助函式
from playsound import playsound
# 撥放音樂
def PlaySound():
    sound_pth = "E:\Media Cabinet\Musics\Musics\dio zawaruto.mp3"
    playsound(sound_pth)

# 5. 使用資料

In [12]:
# 讀取 csv 資料
gesture_df = pd.read_csv(gesture_data_file)

# 切分成訓練以及測試資料
train_df = gesture_df.iloc[::2]
test_df = gesture_df.iloc[1::2]

# 建立資料集
train_dataset = GestureDataset(train_df)
test_dataset = GestureDataset(test_df)

# 建立加載器
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 6. 建立模型

In [13]:
# 建立模型
model = GestureModel().to(device)
model

GestureModel(
  (relu_linear_stack): Sequential(
    (0): Linear(in_features=34, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=4, bias=True)
  )
)

# 7. 訓練模型

In [14]:
# 訓練模型
optimizer = optim_fn(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
    
print("Done!")
PlaySound() # 完成訓練後撥放音樂

Epoch 1
-------------------------------
loss: 155.744431 [   16/ 1768]
loss: 1.591043 [ 1616/ 1768]
Test Error: 
 Accuracy: 61.9%, Avg loss: 17.481135 

Epoch 2
-------------------------------
loss: 12.844677 [   16/ 1768]
loss: 10.246840 [ 1616/ 1768]
Test Error: 
 Accuracy: 61.3%, Avg loss: 15.102308 

Epoch 3
-------------------------------
loss: 26.306850 [   16/ 1768]
loss: 12.334209 [ 1616/ 1768]
Test Error: 
 Accuracy: 64.0%, Avg loss: 5.627311 

Epoch 4
-------------------------------
loss: 5.436070 [   16/ 1768]
loss: 8.172619 [ 1616/ 1768]
Test Error: 
 Accuracy: 49.3%, Avg loss: 7.646345 

Epoch 5
-------------------------------
loss: 4.444322 [   16/ 1768]
loss: 2.220551 [ 1616/ 1768]
Test Error: 
 Accuracy: 63.2%, Avg loss: 6.670976 

Epoch 6
-------------------------------
loss: 6.500128 [   16/ 1768]
loss: 10.022594 [ 1616/ 1768]
Test Error: 
 Accuracy: 53.9%, Avg loss: 4.629816 

Epoch 7
-------------------------------
loss: 6.168066 [   16/ 1768]
loss: 2.302148 [ 1616/